### PROPERTIES AND EXPLORATION OF HIDDEN *REPRESENTATIONS*


MAIN OBJECTIVES: 
- extract hidden representation customising a standard model 
- discuss qualitatively few basic techniques for analysis of *representations* 

The next few blocks restart from Day2 setting up the basics for training the models we are going to inspect later:

- we make sure folders in the course repo are available;
- we load the MNIST dataset and check the data looks as expected;
- we define basic training and test functionalities we will use for learning the weights of our model
- we define standard training hyperparameters 

In [ ]:
# comment if access to the course repo is already available
!git clone https://github.com/AlbertoCazzaniga/MHPC_2022_test

In [ ]:
# comment if access to the course repo is already available
%cd MHPC_2022_test

In [ ]:
# basic imports, load MNIST
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
import numpy as np
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from scripts import mnist #NOTE: might need to customize this

minibatch_size_train = 64
minibatch_size_test = 1000

trainloader, testloader = mnist.get_data(batch_size_train=minibatch_size_test, batch_size_test=minibatch_size_test)

In [ ]:
# a quick look at MNIST images
inputs,labels = next(iter(testloader))

fig=plt.figure(figsize=(15,10))
for i in range(30):
    plt.subplot(5,6,i+1)
    plt.imshow(np.squeeze(inputs[i]),cmap='bone')
    plt.xticks([])
    plt.yticks([])

In [ ]:
#make sure we handle correctly hardware resources depending on availability, and fix seed
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
#fix basic training settings, and fix seed. NOTE: modify as you please!
seed = 33
torch.manual_seed(seed)
input_size=(1,28,28,)
epochs=15
lr=0.01
momentum=0.0   
log_interval=100

In [ ]:
#set up basic train and test functionalities
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
#define convolutional network with
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = ConvNet().to(device)
summary(model,input_size)

In [ ]:
model.eval()
output = model(inputs.to(device))
pred = output.argmax(dim=1, keepdim=True) 
correct = pred.eq(labels.to(device).view_as(pred)).sum().item()
print('acc = {}'.format(correct/inputs.shape[0]) )

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
save=True

In [ ]:
for epoch in range(1, epochs + 1):
    train(model, device, trainloader, optimizer, epoch)
    test(model, device, testloader)
    
if save:
    torch.save(model.state_dict(),"mnist_cn2.pt")

Exercise 1: Basics.
- Define a model ConvNetReps with an additional attribute to extract the hidden representations of the model ConvNet.
- Define an instance of ConvNetReps and load weights obtained from training of ConvNet
- Extract representations for a batch of the testloader, and inspect dimensions

In [ ]:
#define convolutional network with extract method
class ConvNetReps(nn.Module):
    def __init__(self):
        super(ConvNetReps, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
    '''
    Your code here
    def extract(self,...):
        Input: like MNIST image 
        Output: list of 9 tensors each containing a representation 
                of the forward loop -input and output included!-
    '''

In [ ]:
#define instance of ConvNetReps(), load weights, set in evaluation mode
'''
Your code here
'''

In [ ]:
#load a batch of inputs from the testloader
#extract representations using method defined above
#inspect dimensions
'''
Your code here
'''

Exercise 2: Visualisation.

Note that after each convolution each channel can be interpreted as an image. 
- Choose an image and plot the first 20 channels for the first 4 layers. What does it show about the level of abstraction?

"T-distributed Stochastic Neighbor Embedding" [TSNE-sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) allows to project high dimensional data onto 2-d allowing simple visualisation that preserves some of the properties of the original data.
- Use TSNE to inspect the geometry of some layers (e.g. layers after second maxpooling, two last hidden layers).

- Take home message &/or reading exercise: TSNE has strenght and limitations [Distill-TSNE](https://distill.pub/2016/misread-tsne/)

In [ ]:
#visualize channel images
'''
Your code here
'''

In [ ]:
#tsne geometry inspection
from sklearn.manifold import TSNE
'''
Your code here
'''

Exercise 3: Intrinsic dimension

Monitoring the dimension of the representations of a dataset in the hidden layers of a DNN can reveal information about the learning procedure (see for instance [Ansuini et al., 2019](https://proceedings.neurips.cc/paper/2019/file/cfcce0621b49c983991ead4c3d4d3b6b-Paper.pdf) ).

We consider the TWO-NN algorithm for estimating the intrinsic dimension [Facco et al., 2018](https://www.nature.com/articles/s41598-017-11873-y.pdf) of the representations in various hidden layers. The algorithm only uses the distance among the datapoints.

- The ID estimation is as more reliable as more points we consider. Modify the definition of test loader to obtain batches of 5000. Extract representations. 
- Start by the input representation of this batch: compute the distance matrix using SciPy [pdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html) and [squareform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.squareform.html) functions.
- Use the function block_analysis as indicated to compute the decimation curve of ID and the corresponding errors.
- Do the same for other hidden representations (I suggest 3,6,7 -why?-). Plot the curves together and discuss the behaviour of the ID in the hidden representations of the model.

In [ ]:
#import relevant function for ID computation
from intrinsic_dimension import estimate,block_analysis
#import relevant function for distance matrix computation
from scipy.spatial.distance import pdist,squareform

In [ ]:
#extract representation for a batch of the testloader
'''
Your code here
'''

In [ ]:
#compute distance matrix for the input dataset 
#NOTE: need to understand how to compute distance between image tensors (1,28,28),
#using function block_analysis compute ID dimensions (output[0]), 
#error estimates (output[1]) and number of points considered (output[2])
#in the decimation process
'''
Your code here
'''

In [ ]:
#compute ID also for layers 3,6,7 and discuss qualitatively the behaviour
'''
Your code here
'''

Extra Exercise:

- Repeat the analysis for the hidden representations of MNIST obtained through your favourite MLP classifier. Compare qualitative the results to the one obtained with CNN.